<a href="https://colab.research.google.com/github/nh7881/machine_learning_project_class/blob/master/exercise_10/assignment_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torchvision import transforms, datasets
from matplotlib import pyplot as plt
import numpy as np

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,)),  # mean value = 0.1307, standard deviation value = 0.3081
])

In [3]:
data_path = './MNIST'
minibatch_size=64

training_set = datasets.MNIST(root = data_path, train= False, download=True, transform= transform)
testing_set = datasets.MNIST(root = data_path, train= True, download=True, transform= transform)
training_loader = torch.utils.data.DataLoader(training_set,minibatch_size)
testing_loader = torch.utils.data.DataLoader(testing_set,minibatch_size)

Extracting ./MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/MNIST/raw


Extracting ./MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/MNIST/raw



Extracting ./MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/MNIST/raw


Extracting ./MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
print(training_loader.dataset.data.shape)
print(testing_loader.dataset.data.shape)

torch.Size([10000, 28, 28])
torch.Size([60000, 28, 28])


In [13]:
class classification(nn.Module):
    def __init__(self):
        super(classification, self).__init__()
        
        # construct layers for a neural network
        self.classifier1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=20*20),
            nn.ReLU(),
        ) 
        self.classifier2 = nn.Sequential(
            nn.Linear(in_features=20*20, out_features=10*10),
            nn.ReLU(),
        ) 
        self.classifier3 = nn.Sequential(
            nn.Linear(in_features=10*10, out_features=10),
            nn.LogSoftmax(dim=1),
        ) 
        
        
    def forward(self, inputs):                 # [batchSize, 1, 28, 28]
        x = inputs.view(inputs.size(0), -1)    # [batchSize, 28*28]
        x = self.classifier1(x)                # [batchSize, 20*20]
        x = self.classifier2(x)                # [batchSize, 10*10]
        out = self.classifier3(x)              # [batchSize, 10]
        
        return out


In [14]:
criterion = nn.NLLLoss()

In [15]:
learning_rate_value = 0.01
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
classifier = classification().to(device)
optimizer = torch.optim.SGD(classifier.parameters(), lr=learning_rate_value, momentum=0.9)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.9)

In [16]:
test_losses = []
test_accuracies = []
def test() :
    classifier.eval()
    correct = 0
    n = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(testing_loader):
            data, target = data.to(device), target.to(device)
            output = classifier(data)
            loss = criterion(output, target)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            n += len(data)
        print('Test Loss: {:.6f} Correct: {}, ACC : {}'.format(
            loss.item(), correct, correct/n))
        test_losses.append(loss.item())
        test_accuracies.append(correct/n)

In [17]:
epoch_num = 30
losses = []
accuracies = []
for epoch in range(epoch_num) :
    correct = 0
    n = 0
    classifier.train()
    #scheduler.step()
    for batch_idx, (data, target) in enumerate(training_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = classifier(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        n += len(data)
    print('Train Epoch: {} Loss: {:.6f} Correct: {} ACC: {}'.format(
        epoch, loss.item(), correct, correct/n))
    test()
    losses.append(loss.item())
    accuracies.append(correct/n)
    #train  test    batch_size  loss    activ
    #94.7,  91.95   16          ce      sigmoid
    #?      93.7    64          ce      sigmoid
    #97     92.8    64          nll     sigmoid
    #100    94.9    16          nll     sigmoid
    #100    95.5    64          nll     relu

Train Epoch: 0 Loss: 0.127496 Correct: 7743 ACC: 0.7743
Test Loss: 0.144125 Correct: 51952, ACC : 0.8658666666666667
Train Epoch: 1 Loss: 0.072749 Correct: 9149 ACC: 0.9149
Test Loss: 0.070219 Correct: 54153, ACC : 0.90255
Train Epoch: 2 Loss: 0.113736 Correct: 9391 ACC: 0.9391
Test Loss: 0.037383 Correct: 55292, ACC : 0.9215333333333333
Train Epoch: 3 Loss: 0.020197 Correct: 9539 ACC: 0.9539
Test Loss: 0.022966 Correct: 55662, ACC : 0.9277
Train Epoch: 4 Loss: 0.008091 Correct: 9693 ACC: 0.9693
Test Loss: 0.020598 Correct: 56249, ACC : 0.9374833333333333
Train Epoch: 5 Loss: 0.011748 Correct: 9784 ACC: 0.9784
Test Loss: 0.018580 Correct: 56454, ACC : 0.9409
Train Epoch: 6 Loss: 0.002839 Correct: 9859 ACC: 0.9859
Test Loss: 0.016131 Correct: 56658, ACC : 0.9443
Train Epoch: 7 Loss: 0.002788 Correct: 9917 ACC: 0.9917
Test Loss: 0.012397 Correct: 56787, ACC : 0.94645
Train Epoch: 8 Loss: 0.001580 Correct: 9956 ACC: 0.9956
Test Loss: 0.008893 Correct: 56842, ACC : 0.9473666666666667
Train